In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/document_classifier/")

In [ ]:
# !pip install --upgrade scikit-learn

In [ ]:
from sklearn.naive_bayes import MultinomialNB # classifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # text vectorizer
#from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score  # evaluation
import matplotlib.pyplot as plt # visualization
import pandas as pd # data representation
from sklearn.pipeline import make_pipeline
from sklearn.metrics import ConfusionMatrixDisplay
import pandas as pd

In [ ]:
import pandas as pd
prev_data_ = pd.read_csv("./health_data/march_text_data.csv")
magma_data_ = pd.read_csv("./health_data/magma_chola_liberty_data.csv")

prev_data = prev_data_[['text_pdf', 'page_class']]
magma_data = magma_data_[['text_pdf', 'page_class']]
prev_data.rename(columns = {'text_pdf': 'text', 'page_class': 'class'}, inplace=True)
magma_data.rename(columns = {'text_pdf': 'text', 'page_class': 'class'}, inplace=True)

final_data = pd.concat([prev_data, magma_data], axis=0)

<ipython-input-89-0ed46851d989>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_data.rename(columns = {'text_pdf': 'text', 'page_class': 'class'}, inplace=True)
<ipython-input-89-0ed46851d989>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  magma_data.rename(columns = {'text_pdf': 'text', 'page_class': 'class'}, inplace=True)


In [ ]:
final_data.head()

,text,class
0,Auto Secure ­ Private Car Package Policy\nName...,0
1,\n \n^Premium mentioned covers Self and Famil...,0
2,\n TATA AIG General Insurance Company Lt...,0
3,\nCertificate Of Insurance and Policy Schedul...,1
4,Basis of claim\nsettlement for Tyre\nSecure ...,0


In [ ]:
from sklearn.model_selection import train_test_split

train_prev, test_prev = train_test_split(final_data, test_size=0.2)

In [ ]:
train_df_ = pd.read_csv("./health_data/train_metadata_health.csv")
test_df_ = pd.read_csv("./health_data/test_metadata_health.csv")
train_df = train_df_[['text', 'class']]
test_df = test_df_[['text', 'class']]

train_df['class'] = train_df['class'].apply(lambda x: 0 if x=='not_important' else 1)
test_df['class'] = test_df['class'].apply(lambda x: 0 if x=='not_important' else 1)

# train_df = train_df[['text', 'class']]
# test_df = test_df[['text', 'class']]
train_df.head(5)

<ipython-input-92-79ef01a6f4d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['class'] = train_df['class'].apply(lambda x: 0 if x=='not_important' else 1)
<ipython-input-92-79ef01a6f4d8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['class'] = test_df['class'].apply(lambda x: 0 if x=='not_important' else 1)


,text,class
0,Attached to and forming part of Policy No P/17...,0
1,Attached to and forming part of Policy No. P/6...,0
2,IMPORTANT 10-FEB-23 To_ AVINISH KUMAR_ HOUSE N...,0
3,Star Hezlth Personal Cating Insurance Star Hea...,0
4,STAR Health Persona Czring Insurnce Star Healt...,0


In [ ]:
final_train = pd.concat([train_prev, train_df]).reset_index(drop=True)
final_test = pd.concat([test_prev, test_df]).reset_index(drop=True)

In [ ]:
final_train.head()

,text,class
0,Policy Number: 3362/02486310/000/00\n Certif...,1
1,The words and phrases listed have special mean...,0
2,REGISTRATION \nMARK & PLACE\nENGINE NO. & CHAS...,1
3,�����������\nBAJAJ ALLIANZ GENERAL INSURANCE C...,1
4,Vehicle Details\nPolicy Details\nMake\nHyundai...,1


In [ ]:
final_train['class'].value_counts()

0    3254
1    3247
Name: class, dtype: int64

In [ ]:
final_train = final_train[final_train['text'].notnull()]
final_test = final_test[final_test['text'].notnull()]

In [ ]:
final_train['class'].value_counts()

1    3233
0    3232
Name: class, dtype: int64

In [ ]:
## A function to remove punctuations and other symbols
import string

punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    # print(text)
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [ ]:
import nltk
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    tokens = text.split()
    output= " ".join([i for i in tokens if i not in stopwords])
    # print(output)
    return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
final_train['text'] = final_train['text'].apply(lambda x: remove_punctuations(x))
final_train['text'] = final_train['text'].apply(lambda x: str(x).replace("  ", " "))
final_train['text'] = final_train['text'].apply(lambda x: remove_stopwords(x))
final_train['text'] = final_train['text'].apply(lambda x: x.lower())
# final_train['label'] = final_train['class'].apply(lambda x: 0 if x == 'not_important' else 1)
final_train.rename(columns = {'class': 'label'}, inplace=True)
final_test['text'] = final_test['text'].apply(lambda x: remove_punctuations(x))
final_test['text'] = final_test['text'].apply(lambda x: str(x).replace("  ", " "))
final_test['text'] = final_test['text'].apply(lambda x: remove_stopwords(x))
final_test['text'] = final_test['text'].apply(lambda x: x.lower())
# final_test['label'] = final_test['class'].apply(lambda x: 0 if x == 'not_important' else 1)
final_test.rename(columns = {'class': 'label'}, inplace=True)

In [ ]:
# final_train

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

# # Load the saved pipeline checkpoint
# checkpoint_filename = './health_data/finalized_model.pkl'

# with open(checkpoint_filename, 'rb') as f:
#     loaded_pipeline = pickle.load(f)

text_clf = Pipeline([('vect', TfidfVectorizer(min_df=2, ngram_range=(1,4))),
                      ('clf', SVC(class_weight='balanced', gamma='scale', probability=True,
                                           random_state=50)) ])

# You can access the individual steps
# vectorizer = loaded_pipeline.named_steps['vect']
# classifier = loaded_pipeline.named_steps['clf']

# train the model
text_clf.fit(final_train.text, final_train.label)
# Predict the test cases
predicted = text_clf.predict(final_test.text)

from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy as np

print('Accuracy achieved is ' + str(np.mean(predicted == final_test.label)))
print(metrics.classification_report(final_test.label, predicted, target_names=['not_important', 'important'])),
metrics.confusion_matrix(final_test.label, predicted)

Accuracy achieved is 0.97724399494311
               precision    recall  f1-score   support

not_important       0.98      0.97      0.98       798
    important       0.97      0.98      0.98       784

     accuracy                           0.98      1582
    macro avg       0.98      0.98      0.98      1582
 weighted avg       0.98      0.98      0.98      1582



array([[774,  24],
       [ 12, 772]])

In [ ]:
import pickle
# save the model to disk
filename = './health_data/finalized_model_v2.pkl'
pickle.dump(text_clf, open(filename, 'wb'))

# some time later...

# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# !pip install pdf2image
# !sudo apt-get install poppler-utils
# !pip install PyPDF2
# output.clear()

In [ ]:
pdf_path = './a2937214-5676-4354-a85c-8c26580b46ab.pdf'
import PyPDF2, os, re
from pdf2image import convert_from_path
filename = pdf_path.split("/")[-1].split(".")[0]
if not os.path.exists(f"./media/{filename}"):
    os.makedirs(f"./media/{filename}")
images = convert_from_path(pdf_path)
read_pdf = PyPDF2.PdfFileReader(pdf_path)
number_of_pages = read_pdf.getNumPages()
for i, page_number in enumerate(range(number_of_pages)):   # use xrange in Py2
    page = read_pdf.getPage(page_number)
    page_content = page.extractText()
    images[i].save(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.jpg', 'JPEG')
    file1 = open(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.txt',"w")#append mode
    page_content = re.sub(' +', ' ', page_content)
    file1.write(page_content)
    file1.close()

In [ ]:
image_list, txt_list = [], []
for file in os.listdir(f"./media/{filename}/"):
  if file.endswith('.jpg'):
      image_list.append(f"./media/{filename}/" + file)
      f = open(f"./media/{filename}/" + file.replace(".jpg", '.txt'), 'r')
      txt =  f.read()
      txt_list.append(txt)
      f.close()

In [ ]:
# !pip install easyocr

In [ ]:
# import easyocr
# reader = easyocr.Reader(['en'], gpu = True)

In [ ]:
# pdf_path = './b36cf186-41a6-4b56-a644-232b97b50288.pdf'
# import PyPDF2, os, re
# from pdf2image import convert_from_path
# filename = pdf_path.split("/")[-1].split(".")[0]
# if not os.path.exists(f"./media/{filename}"):
#     os.makedirs(f"./media/{filename}")
# images = convert_from_path(pdf_path)
# read_pdf = PyPDF2.PdfFileReader(pdf_path)
# number_of_pages = read_pdf.getNumPages()
# for i, page_number in enumerate(range(number_of_pages)):   # use xrange in Py2
#     # page = read_pdf.getPage(page_number)
#     # page_content = page.extractText()
#     images[i].save(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.jpg', 'JPEG')
#     data_eng = " ".join(reader.readtext(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.jpg', detail = 0, paragraph=True))
#     file1 = open(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.txt',"w") #append mode
#     page_content = re.sub(' +', ' ', data_eng)
#     file1.write(page_content)
#     file1.close()

In [ ]:
import string

punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [ ]:
import nltk
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    tokens = text.split()
    output= " ".join([i for i in tokens if i not in stopwords])
    return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
txt_ = list(map(remove_punctuations, txt_list))
txt_ = list(map(lambda x: str(x).replace("  ", " "), txt_))
txt_ = list(map(remove_stopwords, txt_))
final_txt_list = list(map(lambda x: x.lower(), txt_))

In [ ]:
final_txt_list[0]

'future generali india insurance company limited corp regd office 6thfloor tower 3 indiabulls finance center senapati bapat marg elphinstone road mumbai 400013 maharashtra care line 1800 220 233 1860 500 3333 022 67837800 email fgcare futuregenerali website www futuregenerali irda regn no 132 cin u66030mh2006plc165287 if undelivered please return future generali india insurance company limited 205 to 208 2nd floor aurum opp astha clinic near vasna petrol pump makarand desai road vasna vadodara gujarat 390007 date 30 06 2020 policy no v7283988 mr ashwinbhai arvindbhai bagdai a 9 madhuvan park society near indrapuri karelibaug vadodara gujarat pincode 390018 intermediary name invictus insurance broking services pvt ltd brr fpv to activate road side assistance rsa cover sms activate policy v7283988 to 9222211100 standard sms charges applicable dear mr ashwinbhai arvindbhai bagdai welcome future generali experience wethank youforchoosing motor secure insurance policy your policy no v728398

In [ ]:
predicted = loaded_model.predict(final_txt_list)

In [ ]:
predicted

array([0, 1, 1, 0, 1, 0, 0, 0])

In [ ]:
target_name = ['not_important', 'important']
pred_prob = loaded_model.predict_proba(final_txt_list)
ix = pred_prob.argmax(1)
prediction = [(target_name[val], round(prob[val], 2)) for val, prob in zip(ix, pred_prob)]
print(prediction)

[('not_important', 0.98), ('important', 0.98), ('important', 0.99), ('not_important', 0.89), ('important', 0.99), ('not_important', 1.0), ('not_important', 1.0), ('not_important', 0.97)]


In [ ]:
# for i in range(len(y_pred)):
#     print(f'"{target_categories[y_pred[i]]:<22}" ==> "{text[i]}"')